In [85]:
# Import libraries and set desired options
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import pickle

In [86]:
# Import libraries and set desired options
import pickle
from scipy import sparse
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [4]:
%pwd

'/home/sramirez/git/mlcourse.ai/jupyter_english/assignments_spring2019'

In [87]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [6]:
# Read the training and test data sets, change paths if needed
times = ['time%s' % i for i in range(1, 11)]
train_df = pd.read_csv('../../data/train_sessions.csv',
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv('../../data/test_sessions.csv',
                      index_col='session_id', parse_dates=times)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [88]:
# Load sparse matrix with information about bag of words and date-time features

X_train_new = sparse.load_npz("X_train_new.npz")
X_test_new = sparse.load_npz("X_test_new.npz")

In [89]:
X_train_new

<253561x50008 sparse matrix of type '<class 'numpy.float64'>'
	with 4647358 stored elements in COOrdinate format>

# Freeride (from here down everything is coded by me)

In [90]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


## Attach session duration, mean session diff duration, and other metrics to the matrix

In [91]:

new_vars = ['session_duration', 'mean_intra_duration', 'std_intra_duration', 'n_valid_sessions']
    
def attach_session_vars(df, X_sparse):
    scaled_tmp = StandardScaler().fit_transform(df[new_vars])
    X = hstack([X_sparse, scaled_tmp])
    return X

def generate_session_features(tmp):
    df = tmp.copy()
    times = ['time%s' % i for i in range(1, 11)]
    min_time = df[times].min(axis=1)
    max_time = df[times].max(axis=1)
    session_duration = (max_time - min_time) / np.timedelta64(1, 's')
    df['session_duration'] = session_duration
    mean_intra_duration = df[times].apply(lambda v: v.dropna().diff().mean(), axis=1) / np.timedelta64(1, 's')
    df['mean_intra_duration'] = mean_intra_duration #.clip(quant_df[low], quant_df[high])
    std_intra_duration = df[times].apply(lambda v: v.dropna().diff().std(), axis=1) / np.timedelta64(1, 's')
    df['std_intra_duration'] = std_intra_duration
    n_valid_sessions = df[times].apply(lambda v: v.notnull().sum(), axis=1)
    df['n_valid_sessions'] = n_valid_sessions
    return df[new_vars]

def compute_session_stats(df, X_sparse):
    tmp = generate_session_features(df)
    return attach_session_vars(tmp, X_sparse)

In [92]:
%time

train_df_session = generate_session_features(train_df)
test_df_session = generate_session_features(test_df)


In [93]:
train_df_session.isnull().sum(axis=0)

session_duration          0
mean_intra_duration    3463
std_intra_duration     6642
n_valid_sessions          0
dtype: int64

In [94]:
train_df_session.to_pickle('train_df_session.pkl')
test_df_session.to_pickle('test_df_session.pkl')

In [96]:
# Load train_df_session with NaNs

train_df_session = pd.read_pickle('train_df_session.pkl')
test_df_session = pd.read_pickle('test_df_session.pkl')

In [97]:
%time
#  Fill NaNs accordingly

train_df_session.fillna(0.0, inplace=True)
test_df_session.fillna(0.0, inplace=True)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [98]:
X_sparse_session_tr = attach_session_vars(train_df_session, X_train_new)
X_sparse_session_tst = attach_session_vars(test_df_session, X_test_new)

/usr/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [99]:
sparse.save_npz("X_train_session.npz", X_sparse_session_tr)
sparse.save_npz("X_test_session.npz", X_sparse_session_tst)

In [101]:
X_sparse_session_tr = sparse.load_npz("X_train_session.npz")
X_sparse_session_tst = sparse.load_npz("X_test_session.npz")

In [103]:
X_sparse_session_tr.shape

(253561, 50012)

In [104]:
 X_sparse_session_tr.shape[1] - X_train_new.shape[1] # How many features have been added in this step

4

In [105]:
### MODELING

y_train = train_df['target'].astype('int').values
time_split = TimeSeriesSplit(n_splits=10)
#c_values = np.logspace(-2, 2, 10)
c_values = [0.21544346900318834]
logit = LogisticRegression(C=1, random_state=17, max_iter=1000, solver='liblinear')
logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=4, cv=time_split, verbose=1)

In [106]:
%%time
logit_grid_searcher.fit(X_sparse_session_tr, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   25.3s finished


CPU times: user 5.88 s, sys: 280 ms, total: 6.16 s
Wall time: 31.2 s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'C': [0.21544346900318834]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=1)

In [107]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9124407279026251, {'C': 0.21544346900318834})

In [57]:
# old comparison

logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9124878763993826, {'C': 0.21544346900318834})

In [108]:
pickle.dump(logit_grid_searcher, open("model_LR_session_enrichment_2", 'wb'))

In [109]:
# load the model from disk
logit_grid_searcher = pickle.load(open("model_LR_session_enrichment_2", 'rb'))

In [113]:
# compare with other models

logit_grid_searcher_datetime = pickle.load(open("model_yorko_baseline", 'rb'))
print(logit_grid_searcher_datetime.cv_results_['mean_test_score'])
print(logit_grid_searcher_datetime.cv_results_['mean_train_score'])

[0.88827382 0.90240801 0.91117039 0.91150958 0.91204276 0.90829752
 0.90710592 0.91233321 0.90766483 0.90654529]
[0.95144248 0.96534198 0.97768121 0.98357726 0.98935616 0.99126663
 0.99452571 0.99280502 0.99214336 0.99174035]


In [124]:
train_df.time1.min(), train_df.time1.max()


(Timestamp('2013-01-12 08:05:57'), Timestamp('2014-04-30 23:39:53'))

In [120]:
from 
logit_grid_searcher.best_estimator_.

<bound method TimeSeriesSplit.split of TimeSeriesSplit(max_train_size=None, n_splits=10)>

In [114]:
logit_test_pred5 = logit_grid_searcher.predict_proba(X_sparse_session_tst)[:, 1]
write_to_submission_file(logit_test_pred5, 'subm5_2.csv') # 0.94242 (yorko), with date features we move to 0.94962


In [116]:
logit_grid_searcher.cv_results_

{'mean_fit_time': array([8.27388163]),
 'std_fit_time': array([4.35206477]),
 'mean_score_time': array([0.02261157]),
 'std_score_time': array([0.01068711]),
 'param_C': masked_array(data=[0.21544346900318834],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.21544346900318834}],
 'split0_test_score': array([0.76528412]),
 'split1_test_score': array([0.74674997]),
 'split2_test_score': array([0.95165791]),
 'split3_test_score': array([0.97203986]),
 'split4_test_score': array([0.91033298]),
 'split5_test_score': array([0.95815955]),
 'split6_test_score': array([0.94495116]),
 'split7_test_score': array([0.94467358]),
 'split8_test_score': array([0.96441434]),
 'split9_test_score': array([0.96614381]),
 'mean_test_score': array([0.91244073]),
 'std_test_score': array([0.07998397]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([0.99318555]),
 'split1_train_score': array([0.99116021]),
 'split2_train_score': a

## Remove bias data from X months

## LDA feature extraction